In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
CSV_PATH = '/content/papers.csv'
df = pd.read_csv(CSV_PATH, engine='python')

In [ ]:
df.head()

,Unnamed: 0,title,id,abstract,authors,url
0,0,constraint-preserving hybrid finite element me...,1907.00084,maxwell's equations describe the evolution of ...,"['yakov berchenko-kogan', 'ari stern']",https://arxiv.org/abs/1907.00084
1,1,rewriting structured cospans,2001.09029,to support the study of compositional networks...,['daniel cicala'],https://arxiv.org/abs/2001.09029
2,2,hybridization and postprocessing in finite ele...,2008.00149,we hybridize the methods of finite element ext...,"['gerard awanou', 'maurice fabien', 'johnny gu...",https://arxiv.org/abs/2008.00149
3,3,novel approach to synchronisation of wearable ...,2107.03147,synchronisation of wireless inertial measureme...,"['andreas spilz', 'michael munz']",https://arxiv.org/abs/2107.03147
4,4,functional equivariance and conservation laws ...,2111.10042,preservation of linear and quadratic invariant...,"['robert i. mclachlan', 'ari stern']",https://arxiv.org/abs/2111.10042


In [ ]:
!pip install pytextrank spacy

In [ ]:
import pytextrank
import spacy

# Cohere's embedding model has a 512 token context window
# This is insufficient, as abstract + title is likely much longer
# Perform sentence summarization to get a succinct version within context limit
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank")

TOKENS_PER_WORD = 1.5  # Guesstimate
CONTEXT_WINDOW = 512   # Specific to Cohere's embedding model

texts = df['title'] + ' ' + df['abstract']

trimmed_data = []

for doc in nlp.pipe(texts, batch_size=32):
    tr = doc._.textrank
    tokens = 0
    summary_sentences = []
    for sentence in tr.summary(limit_sentences=10, preserve_order=True):
        summary_sentences.append(sentence.text)
        tokens += TOKENS_PER_WORD * len(sentence.text.split())
        if tokens >= CONTEXT_WINDOW:
            break
    trimmed_data.append(" ".join(summary_sentences))


In [ ]:
!pip install pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=0c385ef1118ff9b36be025309a4fcee8a8d5ec505909aae487a64ffb21f29dfa
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc


In [ ]:
from pylatexenc.latex2text import LatexNodes2Text

# Some LaTeX commands/formatting cause a 400 error when POST'ing to the embedding API
# Parse out LaTeX, leaving only natural language
conv = LatexNodes2Text()
data = []
for latex_doc in trimmed_data:
  try:
    raw_text = conv.latex_to_text(latex_doc)
    data.append(raw_text[:2048]) # API limitation, can only have 2048 chars
  except IndexError as e:
    continue


In [ ]:
import time
import tqdm
import requests
from google.colab import userdata
import json
import numpy as np

MODEL_NAME = 'cohere-embed-multilingual'
BATCH_SIZE = 96 # Maximum supported by Cohere's free API
ENDPOINT = 'https://us.inference.heroku.com/v1/embeddings'
HEADER = {
    "Authorization": f"Bearer {userdata.get('EMBEDDING_KEY')}",
    "Content-Type": "application/json"
}

embeddings = []
for i in tqdm.tqdm(range(0, len(data), BATCH_SIZE)):
  payload = {
          "model": MODEL_NAME,
          "input": data[i:i+BATCH_SIZE],
          "input_type": "search_document",
          "allow_ignored_params": True
  }
  for attempt in range(4): #backoff for rate limits
    try:
      resp = requests.post(url=ENDPOINT, headers=HEADER, json=payload)
      resp.raise_for_status()
      for idx, embedding in enumerate(resp.json()['data']):
        embeddings.append((i+idx, embedding['embedding']))
      break
    except requests.exceptions.HTTPError as e:
      if e.errno == 429:
        time.sleep(2 ** attempt)
      else:
        print(f'Failure in batch {i}')
        break

  with open('embeddings.pkl', 'wb') as fp:
    pickle.dump(embeddings, fp)


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 55.8 MB/s eta 0:00:00


In [ ]:
import pickle

with open('embeddings.pkl', 'rb') as fp:
  embeddings = pickle.load(fp)

In [ ]:
# We'll use FAISS for fast retrieval in kNN
paper_index = faiss.IndexFlatIP(len(embeddings[0]))
paper_index.add(embeddings)
faiss.write_index(paper_index, 'paper_index.faiss')

/tmp/ipython-input-468649182.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(columns=df.columns[0], inplace=True)


(57792, 5)
title       57787
id          57792
abstract    57790
authors     55971
url         57792
dtype: int64


In [ ]:
df2.to_csv('papers_trimmed.csv')